<a href="https://colab.research.google.com/github/Sharon-Tseng/VincentMobile_Database_System/blob/main/ISOM3260_Progress_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer and Member

In [1]:
# -- Copy and paste the whole code when doing project--
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
# create a link
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle

--2025-03-30 12:40:07--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 184.24.36.87
Connecting to download.oracle.com (download.oracle.com)|184.24.36.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M   130MB/s    in 0.6s    

2025-03-30 12:40:08 (130 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: 

In [2]:
# -- Copy and paste the whole code when doing project, using personal account for testing --
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = " IMZ409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "ytsengaa"  # use your own username and password
PASSWORD = "7192"    # use your own username and password

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)

c = conn.cursor()   # c: represent database connection

In [17]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import Label
from datetime import date
import re

In [19]:
from logging import disable
from pickle import FALSE
# tab object
tb_all = TabBar(['Introduction','Register','Login','Welcome','Forget_Password',
          'Change_Password','Update_Personal_Account','Message_toIntro','Message_toWelcome'])

# a simple function for jumping between tabs
def showPage(tabName):
  with tb_all.output_to(tabName):
    display()

##############################################
############## Introduction Tab ###########
##############################################

with tb_all.output_to('Introduction'):
  intro_label = Label("Welcome to 302 Vincent Mobile")

  btn_RegisterPage = widgets.Button(description="Register as Member")
  btn_LoginPage = widgets.Button(description="Login")

  def on_btnRegisterPage_button_clicked(b):
    showPage("Register")
  btn_RegisterPage.on_click(on_btnRegisterPage_button_clicked)

  def on_btn_LoginPage_button_clicked(b):
    showPage("Login")
  btn_LoginPage.on_click(on_btn_LoginPage_button_clicked)

  intro_row1HBox = widgets.HBox([intro_label])
  intro_row2HBox = widgets.HBox([btn_LoginPage])
  intro_row3HBox = widgets.HBox([btn_RegisterPage])

  display(widgets.VBox([intro_row1HBox, intro_row2HBox, intro_row3HBox]))

##############################################
############## Register Tab ###############
##############################################

with tb_all.output_to("Register"):
  reg_label = Label("Please fill in the information")
  username_label = Label("Username: ")
  username_textbox = Text(placeholder='Enter your username', disabled=False)
  password_label = Label("Password: ")
  password_textbox = Text(placeholder='Enter your password', disabled=False)
  firstname_label = Label("First Name: ")
  firstname_textbox = Text(placeholder='Tai Ming', disabled=False)
  lastname_label = Label("Last Name: ")
  lastname_textbox = Text(placeholder='Wong', disabled=False)
  address_label = Label("Address: ")
  address_textbox = Text(placeholder='Sai Kung, Hong Kong', disabled=False)
  gender_label = Label("Gender: ")
  gender_dropdown = widgets.Dropdown(options=["Male", "Female", "Others"], value=None)
  dob_label = Label("Date of Birth: ")
  dob_datepicker = DatePicker(placeholder='2000/01/01', disabled=False)
  phone_label = Label("Phone: ")
  phone_textbox = Text(placeholder='12345678', disabled=False)
  email_label = Label("Email: ")
  email_textbox = Text(placeholder='e.g, abc@defg', disabled=False)
  creditno_label = Label("Credit No.: ")
  creditno_textbox = Text(placeholder='1234 5678 9123 4567', disabled=False)
  security_question_label = Label("Security Question: ")
  cmb = widgets.Dropdown(options=["What is your favorite animal?", "Which city did you live when you were 10?",
                    "What year is your grandmother born?", "What is the name of your first pet?",
                    "What is your favorite sport?"], value = None)
  security_answer_label = Label("Security Answer: ")
  security_answer_textbox = Text(placeholder='Security Answer', disabled=False)
  expiry_date_label = Label("Expiry Date: ")
  expiry_date_datepicker = DatePicker(dplaceholder='1991-01-01', disabled=False)
  cvc_label = Label("CVC: ")
  cvc_textbox = Text(placeholder='000', disabled=False)

  reg_register_button = widgets.Button(description="Register")
  reg_back_button = widgets.Button(description="Back")


  # Register Button - event handler
  def on_register_button_clicked(b):

    # exclude duplicated username
    c.execute("SELECT M_USERNAME FROM MEMBER WHERE M_USERNAME = '{}'".format(username_textbox.value))
    existing_user = c.fetchone()

    if existing_user[0] == username_textbox.value:
      msg_label_1.value = "Username already exists. Please create another username."
      showPage("Message_toIntro")

      username_textbox.value = ""
      password_textbox.value = ""
      firstname_textbox.value = ""
      lastname_textbox.value = ""
      address_textbox.value = ""
      phone_textbox.value = ""
      email_textbox.value = ""
      creditno_textbox.value = ""
      security_answer_textbox.value=""
      cvc_textbox.value=""
      gender_dropdown.value = None
      cmb.value = None
      dob_datepicker.value = None
      expiry_date_datepicker.value = None



    c.execute(
    """
    INSERT INTO MEMBER VALUES(seq_member.nextval, '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}' ,TO_DATE('{}', 'YYYY-MM-DD'), '{}', '{}', '{}', TO_DATE('{}', 'YYYY-MM-DD'), '{}')
    """.format(firstname_textbox.value, lastname_textbox.value,
          username_textbox.value, password_textbox.value, address_textbox.value,
          cmb.value, security_answer_textbox.value, creditno_textbox.value,
          expiry_date_datepicker.value, cvc_textbox.value, phone_textbox.value, email_textbox.value,
          dob_datepicker.value, gender_dropdown.value)
    )

    c.execute("commit")

    username_textbox.value = ""
    password_textbox.value = ""
    firstname_textbox.value = ""
    lastname_textbox.value = ""
    address_textbox.value = ""
    phone_textbox.value = ""
    email_textbox.value = ""
    creditno_textbox.value = ""
    security_answer_textbox.value=""
    cvc_textbox.value=""
    gender_dropdown.value = None
    cmb.value = None
    dob_datepicker.value = None
    expiry_date_datepicker.value = None


    reg_register_button.disabled=True
    msg_label_1.value = "Successfully Registered!"
    showPage("Message_toIntro")
  reg_register_button.on_click(on_register_button_clicked)

  def on_back_button_clicked(b):
    showPage("Introduction")
  reg_back_button.on_click(on_back_button_clicked)

  # display objects
  reg_msg_labelVBox = widgets.VBox([reg_label])
  reg_labelVBox = widgets.VBox([username_label, password_label, firstname_label, lastname_label,
                  address_label,gender_label,dob_label,phone_label,email_label,
                  creditno_label,security_question_label,security_answer_label,expiry_date_label,cvc_label])
  reg_textboxVBox = widgets.VBox([username_textbox, password_textbox, firstname_textbox, lastname_textbox,
                  address_textbox,gender_dropdown,dob_datepicker,phone_textbox,email_textbox,
                  creditno_textbox,cmb,security_answer_textbox,expiry_date_datepicker,cvc_textbox])
  reg_line1HBox = widgets.HBox([reg_msg_labelVBox])
  reg_line2HBox = widgets.HBox([reg_labelVBox, reg_textboxVBox])
  reg_line3HBox = widgets.HBox([reg_register_button, reg_back_button])

  display(widgets.VBox([reg_line1HBox,reg_line2HBox, reg_line3HBox]))

##############################################
############## Login Tab ##################
##############################################

with tb_all.output_to('Login'):
  login_label = Label("Please login with your username and password:")
  login_un_label = Label("Username: ")
  login_un_textbox = Text(placeholder='Enter your username', disabled=False)
  login_pwd_label = Label("Password: ")
  login_pwd_textbox = Text(placeholder='Enter your password', disabled=False)

  login_button = widgets.Button(description="Login")
  forget_pwd_button = widgets.Button(description="Forget Password")
  login_back_button = widgets.Button(description="Back")

  member_username = widgets.Label("")
  member_pwd = widgets.Label("")

  def on_button_login_clicked(b):
    c.execute(
    """
    SELECT M_ID
    FROM MEMBER
    WHERE EXISTS (SELECT M_USERNAME, M_PW FROM MEMBER WHERE M_USERNAME = '{}' AND M_PW = '{}')
    """.format(login_un_textbox.value, login_pwd_textbox.value)
    )

    member_username.value = login_un_textbox.value
    member_pwd.value = login_pwd_textbox.value

    login_un_textbox.value = ""
    login_pwd_textbox.value = ""

    row = c.fetchone()

    if row:
      member_id = row[0]
      msg_label_2.value = "Welcome back!"
      showPage("Message_toWelcome")
    else:
      msg_label_1.value = "Incorrect username or password! Please try again."
      showPage("Message_toIntro")

  login_button.on_click(on_button_login_clicked)

  def on_back_button_clicked(b):
    showPage("Introduction")
  login_back_button.on_click(on_back_button_clicked)

  # design login interface
  login_msg_labelVBox = widgets.VBox([login_label])
  login_labelVBox = widgets.VBox([login_un_label,login_pwd_label])
  login_textboxVBox = widgets.VBox([login_un_textbox,login_pwd_textbox])

  login_line1HBox = widgets.HBox([login_msg_labelVBox])
  login_line2HBox = widgets.HBox([login_labelVBox, login_textboxVBox])
  login_line3HBox = widgets.HBox([login_button, forget_pwd_button, login_back_button])

  display(widgets.VBox([login_line1HBox, login_line2HBox, login_line3HBox]))

  def on_forget_pwd_clicked(b):
    showPage("Forget_Password")
  forget_pwd_button.on_click(on_forget_pwd_clicked)

with tb_all.output_to('Forget_Password'):
  fpwd_label = Label("Please fill in the below information to verify your identity:")
  fpwd_un_textbox = Text(value='', placeholder='Enter your username', description='Username:', disabled=False)
  spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  fpwd_question_label = Label("Security Question")
  fpwd_question_textbox = Text(placeholder='e.g, What is your favorite animal?', disabled = True)
  fpwd_answer_label = Label("Security Question Answer")
  fpwd_ans_textbox = Text(value='',placeholder='e.g, dog', disabled=False)

  retrieve_button = widgets.Button(description="Retrieve")
  back_button = widgets.Button(description="Back")
  confirm_button = widgets.Button(description="Confirm")

  # retrieve button
  def on_retrieve_button_clicked(b):
    c.execute("SELECT M_QUESTION FROM MEMBER WHERE M_USERNAME = '" + fpwd_un_textbox.value + "'")
    for row in c:
      fpwd_question_textbox.value = row[0]
    on_retrieve_button_clicked.disabled = True
  retrieve_button.on_click(on_retrieve_button_clicked)

  def on_confirm_button_clicked(b):
    c.execute(
    """
    SELECT M_ANSWER FROM MEMBER
    WHERE M_USERNAME = '{}' AND M_QUESTION = '{}'
    """.format(fpwd_un_textbox.value,fpwd_question_textbox.value))

    for row in c:
      if fpwd_ans_textbox.value == row[0]:
        showPage('Change_Password')
      else:
        msg_label_1.value = "Incorrect answer! Please try again."
        showPage('Message_toIntro')

    fpwd_un_textbox.value = ""
    fpwd_question_textbox.value = ""
    fpwd_ans_textbox.value= ""

  confirm_button.on_click(on_confirm_button_clicked)


  # back button
  def on_back_button_clicked(b):
    showPage("Login")
  back_button.on_click(on_back_button_clicked)

  line1HBox = widgets.HBox([fpwd_label])
  line2HBox = widgets.HBox([fpwd_un_textbox, retrieve_button])
  line3HBox = widgets.HBox([spacer])
  line4HBox = widgets.HBox([fpwd_question_label, fpwd_question_textbox])
  line5HBox = widgets.HBox([fpwd_answer_label, fpwd_ans_textbox])
  line6HBox = widgets.HBox([back_button, confirm_button])

  display(widgets.VBox([line1HBox,line2HBox,line3HBox,line4HBox,line5HBox,line6HBox]))

##############################################
########## Change PWD Tab ################
##############################################

with tb_all.output_to('Change_Password'):
  cpwd_label = Label("Please fill in the below information to reset your password:")
  cpwd_username_label = Label("Username:")
  cpwd_username_textbox = Text(placeholder="Your username", disabled=False)
  new_pwd_label = Label("New Password:")
  new_pwd_textbox = Text(placeholder='Your new password', disabled=False)
  retype_label = Label("Retype:")
  retype_textbox = Text(placeholder='Retype your new password', disabled=False)

  update_button = widgets.Button(description='Update')
  back_button = widgets.Button(description='Back')

  def on_update_button_clicked(b):
    c.execute(
    """
    UPDATE MEMBER
    SET M_PW = '{}'
    WHERE M_USERNAME = '{}'
    """.format(new_pwd_textbox.value, cpwd_username_textbox.value)
    )
    c.execute("commit")
    msg_label_1.value = "Successfully Updated!"
    showPage("Message_toIntro")
  update_button.on_click(on_update_button_clicked)

  def on_back_button_clicked(b):
    showPage("Login")
  back_button.on_click(on_back_button_clicked)

  line1HBox = widgets.HBox([cpwd_label])
  line2HBox = widgets.HBox([cpwd_username_label,cpwd_username_textbox])
  line3HBox = widgets.HBox([new_pwd_label,new_pwd_textbox])
  line4HBox = widgets.HBox([retype_label,retype_textbox])
  line5HBox = widgets.HBox([update_button, back_button])

  display(widgets.VBox([line1HBox,line2HBox,line3HBox,line4HBox,line5HBox]))

##############################################
############## Welcome Tab ################
##############################################

# Pending Tabs: Search Product, Shopping Cart, Purchase History
with tb_all.output_to('Welcome'):
  welcome_label = Label("Welcome! Start browsing with the following functions:")
  change_info_btn = widgets.Button(description='Manage My Account')
  search_prod_btn = widgets.Button(description='Search Products')
  shopping_cart_btn = widgets.Button(description='Shopping Cart')
  history_btn = widgets.Button(description='Purchase History')

  def on_change_info_btn_click(b):
    showPage('Update_Personal_Account')
  change_info_btn.on_click(on_change_info_btn_click)

  display(widgets.VBox([welcome_label,change_info_btn,search_prod_btn,shopping_cart_btn,history_btn]))

##############################################
#### Update_Personal_Account Tab ##########
##############################################


with tb_all.output_to('Update_Personal_Account'):
  m_id_textbox = Text(description='Member_ID', disabled=True)
  up_username_label = Label("Username: ")
  up_username_textbox = Text(placeholder='Your Username', disabled=True)
  up_password_label = Label("Password: ")
  up_password_textbox = Text(placeholder='Your Password', disabled=False)
  up_firstname_label = Label("First Name: ")
  up_firstname_textbox = Text(placeholder='Tai Ming', disabled=False)
  up_lastname_label = Label("Last Name: ")
  up_lastname_textbox = Text(placeholder='Wong', disabled=False)
  up_address_label = Label("Address: ")
  up_address_textbox = Text(placeholder='Sai Kung, Hong Kong', disabled=False)
  up_gender_label = Label("Gender: ")
  up_gender_dropdown = widgets.Dropdown(options=["Male", "Female", "Others"])
  up_dob_label = Label("Date of Birth: ")
  up_dob_datepicker = DatePicker(placeholder='2000/01/01', disabled=False)
  up_phone_label = Label("Phone: ")
  up_phone_textbox = Text(placeholder='12345678', disabled=False)
  up_email_label = Label("Email: ")
  up_email_textbox = Text(placeholder='e.g, abc@defg', disabled=False)
  up_creditno_label = Label("Credit No.: ")
  up_creditno_textbox = Text(placeholder='1234 5678 9123 4567', disabled=False)
  up_security_question_label = Label("Security Question: ")
  up_cmb = widgets.Dropdown(options=["What is your favorite animal?", "Which city did you live when you were 10?",
                    "What year is your grandmother born?", "What is the name of your first pet?",
                    "What is your favorite sport?"], value = None)
  up_security_answer_label = Label("Security Answer: ")
  up_security_answer_textbox = Text(placeholder='Security Answer', disabled=False)
  up_expiry_date_label = Label("Expiry Date: ")
  up_expiry_date_datepicker = DatePicker(dplaceholder='1991-01-01', disabled=False)
  up_cvc_label = Label("CVC: ")
  up_cvc_textbox = Text(placeholder='000', disabled=False)

  retrieve_btn = widgets.Button(description = "Retrieve Info.")
  spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  update_btn = widgets.Button(description = "Update")
  back_btn = widgets.Button(description = "Back")

  # define retrieve info
  def on_retrieve_button_clicked(b):
    c.execute("SELECT * FROM MEMBER WHERE M_USERNAME= '{}' AND M_PW = '{}'".format(member_username.value, member_pwd.value))
    for row in c:
      m_id_textbox.value = str(row[0])
      up_username_textbox.value = row[3]
      up_password_textbox.value = row[4]
      up_firstname_textbox.value = row[1]
      up_lastname_textbox.value = row[2]
      up_address_textbox.value = row[5]
      up_gender_dropdown.value = row[14]
      up_dob_datepicker.value = row[13].date()
      up_phone_textbox.value = row[11]
      up_email_textbox.value = row[12]
      up_creditno_textbox.value = row[8]
      up_cmb.value = row[6]
      up_security_answer_textbox.value = row[7]
      up_expiry_date_datepicker.value =row[9].date()
      up_cvc_textbox.value = row[10]

  retrieve_btn.on_click(on_retrieve_button_clicked)

  # define update info
  def on_update_btn_clicked(b):
    c.execute(
    """
    Update MEMBER
    SET M_USERNAME = '{}',
    M_PW = '{}',
    M_FIRST_NAME = '{}',
    M_LAST_NAME = '{}',
    M_ADDRESS = '{}',
    M_GENDER = '{}',
    M_DOB = TO_DATE('{}', 'YYYY-MM-DD'),
    M_PHONE = '{}',
    M_EMAIL = '{}',
    M_CREDIT_NO = '{}',
    M_QUESTION = '{}',
    M_ANSWER = '{}',
    M_CREDIT_EXPIRE = TO_DATE('{}', 'YYYY-MM-DD'),
    M_CREDIT_CVC = '{}'
    WHERE M_ID = {}
    """.format(up_username_textbox.value,up_password_textbox.value,up_firstname_textbox.value,up_lastname_textbox.value,up_address_textbox.value,up_gender_dropdown.value,
          up_dob_datepicker.value,up_phone_textbox.value,up_email_textbox.value,up_creditno_textbox.value,up_cmb.value,
          up_security_answer_textbox.value,up_expiry_date_datepicker.value,up_cvc_textbox.value,m_id_textbox.value)
    )
    c.execute("commit")

    msg_label_2.value = "Successfully Updated!"
    showPage("Message_toWelcome")

    m_id_textbox.value = ""
    up_username_textbox.value = ""
    up_password_textbox.value = ""
    up_firstname_textbox.value = ""
    up_lastname_textbox.value = ""
    up_address_textbox.value = ""
    up_phone_textbox.value = ""
    up_email_textbox.value = ""
    up_creditno_textbox.value = ""
    up_security_answer_textbox.value=""
    up_cvc_textbox.value=""
    up_gender_dropdown.value = None
    up_cmb.value = None
    up_dob_datepicker.value = None
    up_expiry_date_datepicker.value = None

  update_btn.on_click(on_update_btn_clicked)

  def on_back_btn_clicked(b):
    showPage("Login")
  back_btn.on_click(on_back_button_clicked)


  line1HBox = widgets.HBox([m_id_textbox, retrieve_btn])
  line_label_VBox = widgets.VBox([up_username_label, up_password_label, up_firstname_label, up_lastname_label,
                  up_address_label,up_gender_label,up_dob_label,up_phone_label,up_email_label,
                  up_creditno_label,up_security_question_label,up_security_answer_label,up_expiry_date_label,up_cvc_label])
  line_textbox_VBox = widgets.VBox([up_username_textbox, up_password_textbox, up_firstname_textbox, up_lastname_textbox,
                  up_address_textbox,up_gender_dropdown,up_dob_datepicker,up_phone_textbox,up_email_textbox,
                  up_creditno_textbox,up_cmb,up_security_answer_textbox,up_expiry_date_datepicker,up_cvc_textbox])
  line2_HBox = widgets.HBox([line_label_VBox,line_textbox_VBox])
  line3_HBox = widgets.HBox([back_btn, update_btn])

  display(widgets.VBox([line1HBox,spacer,line2_HBox,line3_HBox]))



##############################################
############## Message Box ################
##############################################

## Message_toIntro ##
with tb_all.output_to('Message_toIntro'):
  msg_label_1 = Label("")
  msg_ok_btn = widgets.Button(description="OK")

  def on_msgbox_click(b):
    showPage("Introduction")
  msg_ok_btn.on_click(on_msgbox_click)

  msg_row1HBox = widgets.HBox([msg_label_1])
  msg_row2HBox = widgets.HBox([msg_ok_btn])

  display(widgets.VBox([msg_row1HBox, msg_row2HBox]))

## Message_toWelcome ##
with tb_all.output_to('Message_toWelcome'):
  msg_label_2 = Label("")
  msg_ok_btn = widgets.Button(description="OK")

  def on_msgbox_click(b):
    showPage("Welcome")
  msg_ok_btn.on_click(on_msgbox_click)

  msg_row1HBox = widgets.HBox([msg_label_2])
  msg_row2HBox = widgets.HBox([msg_ok_btn])

  display(widgets.VBox([msg_row1HBox, msg_row2HBox]))

showPage("Introduction")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>